In [ ]:
!pip install textacy --quiet
!pip install rake_nltk --quiet

import spacy
import string
import textacy
import itertools
from nltk import pos_tag
from rake_nltk import Rake
from nltk import sent_tokenize
from nltk import word_tokenize
from nltk import tree2conlltags
from gensim.summarization import keywords
from nltk.chunk.regexp import RegexpParser
from nltk.corpus.reader.plaintext import PlaintextCorpusReader

In [ ]:
PATH = '/content/drive/MyDrive/DSI Month 4/Week 13/NLP text sets/cnn_lite'

DOC_PATTERN = r'.*\.txt'
corpus = PlaintextCorpusReader(PATH, DOC_PATTERN)

In [ ]:
docs = [corpus.raw(fileid) for fileid in corpus.fileids()]

In [ ]:
doc = docs[0]

In [ ]:
doc

'Des Moines, Iowa (CNN) - Rep. Alexandria Ocasio-Cortez capped her debut on the Iowa hustings in support of Bernie Sanders last week with a blunt call to action. "This is not about something that we allow to happen to us. We don\'t let this race happen to us," she said at a rally for the Vermont senator in Council Bluffs. "We don\'t watch the presidential race. This is not a movie, this a movement." And yet, there has been a certain cinematic quality to the last six weeks of Sanders\' second Democratic presidential campaign. Nearly sidelined, or worse, by a heart attack in Las Vegas on the first night of October, Sanders has charted a remarkable revival. It\'s been powered by a run of invigorating endorsements, new poll results that showed him gaining steam in New Hampshire and Iowa, and the sense, fueled in part by the massive crowds that welcomed him during recent rallies in New York and Minnesota, that his "political revolution" was, after a trying summer, back on the march. There a

In [ ]:
# uses the Textrank algorithm to extract and score keywords
keywords(doc, words=10, lemmatize=True).split('\n')

['sanders',
 'campaign',
 'news',
 'democrats',
 'recent',
 'rep',
 'progressive',
 'cnn',
 'iowa',
 'mayor']

In [ ]:
from rake_nltk import Rake

r = Rake()
r.extract_keywords_from_text(doc)
r.get_ranked_phrases_with_scores()[:5]

[(64.35555555555555,
  'win ," campaign manager faiz shakir told cnn last month'),
 (52.0, 'billionaire former new york city mayor michael bloomberg'),
 (33.0, 'former vice president joe biden came'),
 (24.83888888888889, 'tell donald trump ," sanders said'),
 (23.333333333333332, '© 2019 cable news network')]

In [ ]:
def get_ranked_phrases(doc, n):
  r = Rake()
  r.extract_keywords_from_text(doc)

  return r.get_ranked_phrases_with_scores()[:n]

In [ ]:
key_phrases = [get_ranked_phrases(doc, 5) for doc in docs]

In [ ]:
key_phrases[0]

[(64.35555555555555,
  'win ," campaign manager faiz shakir told cnn last month'),
 (52.0, 'billionaire former new york city mayor michael bloomberg'),
 (33.0, 'former vice president joe biden came'),
 (24.83888888888889, 'tell donald trump ," sanders said'),
 (23.333333333333332, '© 2019 cable news network')]

In [ ]:
import spacy
!python -m spacy download en_core_web_sm

In [ ]:
nlp = spacy.load('en_core_web_sm')

spacy_doc = nlp(doc)

In [ ]:
type(spacy_doc)

spacy.tokens.doc.Doc

In [ ]:
for sent in spacy_doc.sents:
  print(sent)
  print('----'*10)

Des Moines, Iowa (CNN) -
----------------------------------------
Rep. Alexandria Ocasio-Cortez capped her debut on the Iowa hustings in support of Bernie Sanders last week with a blunt call to action.
----------------------------------------
"This is not about something that we allow to happen to us.
----------------------------------------
We don't let this race happen to us," she said at a rally for the Vermont senator in Council Bluffs.
----------------------------------------
"We don't watch the presidential race.
----------------------------------------
This is not a movie, this a movement."
----------------------------------------
And yet, there has been a certain cinematic quality to the last six weeks of Sanders' second Democratic presidential campaign.
----------------------------------------
Nearly sidelined, or worse, by a heart attack in Las Vegas on the first night of October, Sanders has charted a remarkable revival.
----------------------------------------
It's been pow

In [ ]:
# entity extraction
for entity in spacy_doc.ents:
  print(entity.text, '-', entity.label_)

In [ ]:
entities = [[entity.text, entity.label_] for entity in spacy_doc.ents]
entities

[['Des Moines', 'GPE'],
 ['Iowa', 'GPE'],
 ['CNN', 'ORG'],
 ['Alexandria Ocasio-Cortez', 'PERSON'],
 ['Iowa', 'GPE'],
 ['Bernie Sanders', 'PERSON'],
 ['last week', 'DATE'],
 ['Vermont', 'GPE'],
 ['Council Bluffs', 'ORG'],
 ['the last six weeks', 'DATE'],
 ['Sanders', 'ORG'],
 ['second', 'ORDINAL'],
 ['Democratic', 'NORP'],
 ['Las Vegas', 'GPE'],
 ['the first night of October', 'TIME'],
 ['Sanders', 'ORG'],
 ['New Hampshire', 'GPE'],
 ['Iowa', 'GPE'],
 ['New York', 'GPE'],
 ['Minnesota', 'GPE'],
 ['a trying summer', 'DATE'],
 ['march', 'DATE'],
 ['Sanders', 'ORG'],
 ['Saturday', 'DATE'],
 ['Des Moines', 'GPE'],
 ['three', 'CARDINAL'],
 ['Drake University', 'ORG'],
 ['one', 'CARDINAL'],
 ['Ocasio-Cortez', 'ORG'],
 ['Ilhan Omar', 'PERSON'],
 ['Rashida Tlaib', 'PERSON'],
 ['last month', 'DATE'],
 ['Ohio', 'GPE'],
 ['Sanders', 'ORG'],
 ['months', 'DATE'],
 ['Democratic', 'NORP'],
 ['Sanders', 'ORG'],
 ['About two weeks', 'DATE'],
 ['CNN', 'ORG'],
 ['Democrats', 'NORP'],
 ['New Hampshire', '

In [ ]:
from spacy import displacy

displacy.render(spacy_doc, style='ent', jupyter=True)

In [ ]:
import textacy

results = []

for sent in spacy_doc.sents:
  svo = textacy.extract.subject_verb_object_triples(sent)

  results += svo

In [ ]:
# de-duplicate our svo results
results = list(set(results))

[(candidate, entered, race),
 (CNN poll, showed, Sanders),
 (Sanders, delivered, punchline),
 (I, use, profane word),
 (Biden, offers, contrast),
 (he, 's taken, tone),
 (Sanders, decided, to stick),
 (manager Faiz Shakir, told, CNN),
 (one, touched, digits),
 (we, tell, Trump),
 (Iowa caucuses, kick, primary),
 (we, tell, Trump),
 (aide, told, CNN),
 (pugilist, welcomed, news),
 (Sanders, invited, television cameras),
 (We, don't watch, race),
 (Sanders, remains, front),
 (Sanders, has charted, revival),
 (appeal wane, to play, foil),
 (I, thank, you),
 (I, want, to say),
 (I, know, that),
 (surveys, show, group jockeying),
 (me, say, it),
 (he, fair, option),
 (pugilist, welcomed, run),
 (we, allow, to happen),
 (staff, are saying, much),
 (that, has vaulted, him),
 (Sanders, entered, primary),
 (Mayor Pete Buttigieg, has emerged, foe),
 (Sanders, invited, reporters),
 (Sanders, welcomed, news),
 (me, say, it),
 (he, expand, base),
 (campaign, attracting, class coalition),
 (supporte